In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/content/hackathon_data_basketball_2023.csv')

In [22]:
data.head()

,player_defMatchup_a,player_off_a,player_off_ht,player_defMatchup_ht,poss,ptsScored_team,fga,fg,fga3,fg3,...,assistOppCreated,touches,touches_direct,touches_catchAndShoot,touches_paint,touches_time,tov,fta,ft_player,fouls
0,"Lapointe, Uriah","Rinhart, Jordan",82,81,1,0,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,10.12,1.0,NaN,NaN,NaN
1,"Lapointe, Uriah","Weeks, Robert",74,81,1,3,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,6.08,NaN,NaN,NaN,NaN
2,"Lapointe, Uriah","Polson, Aiden",81,81,1,0,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,1.28,NaN,NaN,NaN,NaN
3,"Lapointe, Uriah","Smith, Aaron",80,81,1,0,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,10.28,NaN,NaN,NaN,NaN
4,"Lapointe, Uriah","Garcia, Daniel",82,81,1,2,1.0,1.0,0.0,0.0,...,0.0,2.0,1.0,0.0,1.0,1.76,NaN,NaN,NaN,NaN


## Nading, Samuel

In [54]:
# fill na
data_full = data.fillna(0)
# filter by minimum poss
player_poss = data_full.groupby(by = ['player_off_a']).sum().reset_index()['poss'] > 500

In [95]:
# filter by a poss minimum
player_data = data_full.groupby(by = ['player_off_a']).mean().reset_index().fillna(0)
player_data_filtered = player_data[player_poss]
names = player_data_filtered['player_off_a']

In [102]:
# normalize data
normal_player_data = (player_data_filtered - player_data_filtered.mean()) / player_data_filtered.std()
normal_player_data['player_name'] = names
normal_player_data.drop(columns = ['player_off_a', 'poss'], inplace = True)

<ipython-input-102-b1f4ef67ce87>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  normal_player_data = (player_data_filtered - player_data_filtered.mean()) / player_data_filtered.std()


In [108]:
# get Nading data
nading_data = normal_player_data[normal_player_data['player_name'] == 'Nading, Samuel'].drop(columns = ['player_name'])
nading_data = nading_data.to_numpy()

In [118]:
# calculate similarity to nading
nading_dist = normal_player_data.drop(columns = ['player_name']).apply(lambda x: np.linalg.norm(x-nading_data[0]), 1)
# min-max normalization on the distances
nading_dist = 1 - (nading_dist - nading_dist.min()) / (nading_dist.max() - nading_dist.min())
normal_player_data['nading_dist'] = nading_dist
# 10 most similar players
normal_player_data.sort_values(by = "nading_dist", ascending= False).head(10)

,assistOppCreated,fg,fg3,fga,fga3,fouls,ft_player,fta,iso_actions,iso_pts,...,ptsScored_team,qSQ,touches,touches_catchAndShoot,touches_direct,touches_paint,touches_time,tov,player_name,nading_dist
306,2.672058,2.663032,0.493725,2.493044,1.069680,3.629450,3.087627,3.755852,3.483034,3.485852,...,1.069620,2.321908,2.243324,-1.452349,3.031785,-0.582882,3.152298,2.217990,"Nading, Samuel",1.000000
384,1.870123,2.239327,2.535048,2.046875,2.382188,2.593652,3.503945,2.840881,2.332725,2.761504,...,1.558968,2.228988,1.868179,-0.649080,2.465587,-0.766789,2.217457,1.570135,"Sandoval, Joshua",0.691137
322,1.558736,1.772459,0.133204,1.702877,-0.103268,1.629534,1.745313,1.600622,2.217069,2.193860,...,0.657748,1.419968,2.014259,-1.045424,1.918776,-0.851049,2.944925,0.599161,"Oscarson, Jeremy",0.690100
168,1.236502,2.250871,-0.993394,1.632358,-1.030080,2.514548,3.384089,2.775407,4.238166,4.624963,...,1.357816,1.623960,0.877988,-1.399392,1.706013,-0.537830,1.808179,0.890410,"Hammond, Brandyn",0.688132
430,2.333713,1.608214,0.237691,2.107478,0.215695,2.965719,2.505396,3.004356,0.870596,0.814162,...,-0.220611,2.082609,1.921473,-1.148102,2.779199,-0.702947,2.867546,1.884630,"Thin-Elk, Edward",0.666221
216,1.944045,2.412363,0.473250,2.198088,0.906890,0.887273,0.658542,0.738208,2.241573,1.842161,...,1.143515,2.297959,1.303348,-0.410782,2.119317,-0.126211,1.686775,1.219076,"Kierstead, Iaasic",0.661514
20,1.321348,2.038772,-0.900791,2.002620,-0.631881,3.827451,3.317901,4.017191,3.402305,3.355120,...,-0.144149,2.483631,1.508407,-1.158954,2.278410,0.907191,1.104693,2.659534,"Barnes, Alex",0.634898
378,2.631655,1.173051,-0.087476,1.593884,0.325706,1.731192,2.305395,1.989894,1.345991,1.258588,...,0.193047,1.573178,1.312443,-1.459163,2.485425,-0.831783,2.507589,2.896097,"Rosser, Nathaniel",0.629958
89,2.727990,0.054010,0.884487,0.433578,1.031481,1.142570,1.435180,1.286171,1.952930,1.942955,...,1.384283,0.201050,2.460869,-1.031477,2.245440,-0.797433,2.764674,2.088747,"Cruz, Julian",0.627147
124,0.632545,1.399217,-1.140824,1.468775,-1.244961,2.363372,2.398273,2.268198,2.048921,2.108354,...,0.201079,0.821189,0.213728,-0.554783,1.337421,-0.761272,0.993838,1.233866,"Fourhorn, Nicholas",0.621195
